In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap
from scipy import sparse
from scipy.io import loadmat
from scipy.signal import convolve
from skimage import measure
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.metrics import silhouette_score
from sklearn.preprocessing import StandardScaler

# Initialize random number generator
np.random.seed(42)

In [2]:
# Functions for input data (Caiman, Suite2p, synthetic)

def cnmf_data_to_standard(A_or, timecourse):
    """Convert CaImAn CNMF data to standardized format."""
    if sparse.issparse(A_or):
        A_or = A_or.toarray()
    
    n_pixels = A_or.shape[0]
    fov_size = int(np.sqrt(n_pixels))  # Assuming square FOV
    n_cells = A_or.shape[1]
    
    A_or_full = A_or.reshape((fov_size, fov_size, n_cells), order='F')
    labelimage = np.zeros((fov_size, fov_size))
    
    for n in range(n_cells):
        mask = A_or_full[..., n] > 0
        labelimage[mask] = n + 1
        
    # Handle ROI overlap
    unique_labels = np.unique(labelimage[labelimage > 0])
    missing_cells = set(range(1, n_cells + 1)) - set(unique_labels)
    
    for c in missing_cells:
        cell_mask = A_or_full[..., c-1] > 0
        props = measure.regionprops(cell_mask.astype(int))
        if props:
            y, x = map(int, props[0].centroid)
            labelimage[y, x] = c
    
    return {'timecourse': timecourse, 'labelimage': labelimage}

def generate_synthetic_data(n_neurons=100, n_timepoints=3000, n_clusters=5, fov_size=512):
    """Generate synthetic data in standardized format."""
    neurons_per_cluster = n_neurons // n_clusters
    labelimage = np.zeros((fov_size, fov_size))
    
    # Generate spatial layout
    cluster_centers = np.array([
        [0.2, 0.2],  # Top left
        [0.8, 0.2],  # Top right
        [0.5, 0.5],  # Center
        [0.2, 0.8],  # Bottom left
        [0.8, 0.8]   # Bottom right
    ]) * fov_size
    
    cell_radius = int(fov_size * 0.015)  # Scale with FOV
    cell_id = 1
    
    for cluster in range(n_clusters):
        center = cluster_centers[cluster]
        radius = fov_size * 0.15
        theta = 2 * np.pi * np.random.rand(neurons_per_cluster)
        r = radius * np.sqrt(np.random.rand(neurons_per_cluster))
        
        x = center[0] + r * np.cos(theta)
        y = center[1] + r * np.sin(theta)
        
        for i in range(neurons_per_cluster):
            if all(coord > cell_radius and coord < fov_size - cell_radius 
                  for coord in [x[i], y[i]]):
                XX, YY = np.meshgrid(np.arange(fov_size), np.arange(fov_size))
                mask = ((XX - x[i])**2 + (YY - y[i])**2 <= cell_radius**2)
                if not np.any(labelimage[mask]):
                    labelimage[mask] = cell_id
                    cell_id += 1
    
    # Generate temporal patterns
    t = np.arange(n_timepoints)
    timecourse = np.zeros((n_neurons, n_timepoints))
    frequencies = np.linspace(0.002, 0.006, n_clusters)
    amplitudes = np.linspace(1.5, 2.5, n_clusters)
    
    for i in range(n_clusters):
        period = n_timepoints // n_clusters
        start = i * period
        end = start + period
        
        pattern = amplitudes[i] * np.sin(2 * np.pi * frequencies[i] * (t - start))
        onset = 1 / (1 + np.exp(-(t - start) / 100))
        offset = 1 / (1 + np.exp((t - end) / 100))
        pattern = pattern * onset * offset
        
        # Add cluster variations
        if i % 2 == 0:
            pattern += 0.5 * np.sin(2 * np.pi * 0.05 * (t - start))
        else:
            ramp = np.clip((t - start) / period, 0, 1)
            pattern *= (1 + ramp * 0.5)
            
        pattern = convolve(pattern, np.exp(-np.arange(50)/5), mode='same')
        
        cluster_neurons = slice(i * neurons_per_cluster, (i + 1) * neurons_per_cluster)
        for j, idx in enumerate(range(*cluster_neurons.indices(n_neurons))):
            timecourse[idx] = (pattern + 
                             0.1 * np.sin(2 * np.pi * 0.001 * t + j) + 
                             0.05 * np.random.randn(n_timepoints))
    
    return {'timecourse': timecourse, 'labelimage': labelimage}

In [3]:
DataType = 'synthetic'  # 'synthetic', 'suite2p', 'caiman'
if DataType == 'synthetic':
    # Can specify FOV size if needed
    data = generate_synthetic_data(n_neurons=100, n_timepoints=3000, n_clusters=5, fov_size=512)
else: 
    input_data = loadmat('M000025_ori018_timecourse.mat')
    if DataType == 'caiman':
        data = cnmf_data_to_standard(input_data['A_or'], input_data['timecourse'])
    # else:
    #     data = suite2p_data_to_standard(input_data['stat'], input_data['timecourse'])

timecourse = data['timecourse']
labelimage = data['labelimage']

In [ ]:
# Plot results
plt.figure(figsize=(14, 6))

# Subplot 1: Cell locations
plt.subplot(1, 2, 1)
plt.imshow(labelimage, cmap='nipy_spectral')
plt.title('Cell Locations')
plt.colorbar(label='Cell ID')

# Subplot 2: Example time courses for each cluster
plt.subplot(1, 2, 2)
t = np.arange(timecourse.shape[1])
for i in range(timecourse.shape[0]):
    plt.plot(t, timecourse[i, :], label=f'Neuron {i + 1}')
plt.title('Example Time Courses for Each Cluster')
plt.xlabel('Time')
plt.ylabel('Activity')
handles, labels = plt.gca().get_legend_handles_labels()
plt.legend(handles[:10], labels[:10])

plt.tight_layout()
plt.show()

In [4]:
# 1) Calculate correlation coefficients
# Compute pairwise correlations between all cells
corr_matrix = np.corrcoef(timecourse)

# Determine optimal number of clusters using both Elbow method and Silhouette score
k_range = range(2, 21)  # Test cluster numbers from 2 to 20
# elbow_values = []
silhouette_values = []

# Iterate through different numbers of clusters to find optimal k
for k in k_range:
    # Perform k-means clustering with 10 different random initializations
    kmeans = KMeans(n_clusters=k, init='k-means++', n_init=10)
    labels = kmeans.fit_predict(corr_matrix)
    
    # Calculate metrics for determining optimal k
    # elbow_values.append(kmeans.inertia_)  # Sum of within-cluster distances for Elbow method
    silhouette_values.append(silhouette_score(corr_matrix, labels))  # Average silhouette score

# 2) Perform k-means with optimal k and 10 different random initializations
# Choose k with highest silhouette score
best_k_idx = np.argmax(silhouette_values)
k_optimal = k_range[best_k_idx]
kmeans = KMeans(n_clusters=k_optimal, init='k-means++', n_init=10, random_state=42)
idx_kmeans = kmeans.fit_predict(corr_matrix)

# Store all clustering results in a dictionary
clustering_results = {
    'kmeans': idx_kmeans,
}

# 3) Reorder correlation matrix based on cluster assignments
# Reorder for k-means results
sort_idx_kmeans = np.argsort(idx_kmeans)
sorted_corr_matrix_kmeans = corr_matrix[sort_idx_kmeans][:, sort_idx_kmeans]

# 4) Mean time course for each cluster
unique_clusters = np.unique(idx_kmeans)
n_clusters = len(unique_clusters)
colors = plt.cm.jet(np.linspace(0, 1, n_clusters))
custom_cmap = ListedColormap(colors)

cluster_averages = []
for cluster in unique_clusters:
    cluster_mask = idx_kmeans == cluster  # This creates boolean mask of shape (n_cells,)
    cluster_avg = np.mean(timecourse[cluster_mask], axis=0)  # Average of selected cells
    cluster_averages.append(cluster_avg)

# 5) Cluster maps
cluster_map = np.zeros_like(labelimage)
for cell_idx in range(1, int(np.max(labelimage)) + 1):
    if cell_idx <= len(idx_kmeans):
        roi_mask = labelimage == cell_idx
        cluster_map[roi_mask] = idx_kmeans[cell_idx - 1] + 1  # +1 so clusters start at 1 not 0


In [ ]:
# Visualization should be:
plt.figure(figsize=(15, 5))
# 1. Correlation matrix
plt.subplot(1, 3, 1)
plt.imshow(sorted_corr_matrix_kmeans, cmap='jet')
plt.colorbar()
plt.title(f'k-means Clustering (k = {k_optimal})')
plt.xlabel('Cells')
plt.ylabel('Cells')

save_thumbnails()

# 2. Average time courses
plt.subplot(1, 3, 2)
for i, avg in enumerate(cluster_averages):
    plt.plot(avg, color=colors[i], label=f'Cluster {i+1}')
plt.title(f'Mean time course of each cluster')
plt.xlabel('Time')
plt.ylabel('Cluster average')
plt.legend()

# 3. Cell maps
plt.subplot(1, 3, 3)
plt.imshow(cluster_map, cmap=custom_cmap)
plt.colorbar(label='Cluster')
plt.title('Cluster assignments')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

# Perform PCA
pca = PCA()
score = pca.fit_transform(timecourse.T)  # Transpose to match MATLAB's orientation
coeff = pca.components_.T  # Get eigenvectors (components)
explained = pca.explained_variance_ratio_ * 100  # Convert to percentage

# Components to visualize
num_components = min(timecourse.shape[0], 50)
num_components = 2  # Use 2 components for development

# Setup visualization
fig = plt.figure(figsize=(15, 10))

# Plot spatial maps and temporal components side by side
for i in range(num_components):
    # Calculate contribution map
    contribution = np.abs(coeff[:, i])
    contribution_map = np.zeros_like(labelimage, dtype=float)
    
    # Fill in contribution map
    for cell_idx in range(1, int(np.max(labelimage)) + 1):
        contribution_map[labelimage == cell_idx] = contribution[cell_idx - 1]
    
    # Create subplot for spatial map
    ax = plt.subplot(num_components, 2, 2 * i + 1)  # Odd indices: 1, 3, 5, ...
    im = ax.imshow(contribution_map, cmap='jet')
    plt.colorbar(im, ax=ax)
    ax.set_title(f'PC {i+1} Spatial Map')
    ax.axis('off')
    
    # Create subplot for temporal component
    ax = plt.subplot(num_components, 2, 2 * i + 2)  # Even indices: 2, 4, 6, ...
    ax.plot(score[:, i])
    ax.set_xlabel('Time')
    ax.set_ylabel(f'PC {i+1}')
    ax.set_title(f'PC {i+1} Temporal Profile')

plt.tight_layout()

# Save the figure with subplots
output_filename = "pca_2by2_subplots.png"  # You can change the file format (e.g., .pdf, .svg)
plt.savefig(output_filename, dpi=300, bbox_inches='tight')
print(f"Figure saved as {output_filename}")

# Create new figure for explained variance
plt.figure(figsize=(8, 6))
plt.plot(explained[:10], '-o')
plt.xlabel('PC Component')
plt.ylabel('Variance Explained (%)')
plt.title('Explained Variance')
plt.grid(True)

plt.show()

# Save the explained variance plot
variance_filename = "explained_variance.png"
plt.savefig(variance_filename, dpi=300, bbox_inches='tight')
print(f"Explained variance plot saved as {variance_filename}")

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from matplotlib.colors import ListedColormap

# Save thumbnail function
def save_thumbnail(plot_file, thumbnail_height=300):
    with Image.open(plot_file) as img:
        w, h = img.size
        new_width = int(w * (thumbnail_height / h))
        thumb_img = img.resize((new_width, thumbnail_height), Image.Resampling.LANCZOS)
        thumb_img.save(plot_file.replace(".png", ".thumb.png"))

# Updated PCA visualization function
def generate_pca_visualization(scores, explained_variance, components, roi_masks, output_dir):
    """
    Generate standalone PCA visualizations for each component
    """
    num_components = min(3, components.shape[0], scores.shape[1])
    
    # Plot explained variance
    plt.figure(figsize=(8, 5))
    num_display = min(10, len(explained_variance))
    bars = plt.bar(range(1, num_display + 1), explained_variance[:num_display])
    plt.title("Explained Variance")
    plt.xlabel("Principal Component")
    plt.ylabel("Explained Variance (%)")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    explained_var_path = os.path.join(output_dir, "pca_explained_variance.png")
    plt.savefig(explained_var_path)
    plt.close()
    save_thumbnail(explained_var_path)
    print(f"Explained variance plot saved to {explained_var_path}")

    # Plot each component separately
    for i in range(num_components):
        # Time course plot
        plt.figure(figsize=(8, 5))
        plt.plot(scores[:, i], linewidth=2)
        plt.title(f"PC {i+1} Time Course")
        plt.xlabel("Time")
        plt.ylabel("Component Value")
        plt.grid(True, alpha=0.3)
        plt.tight_layout()
        time_course_path = os.path.join(output_dir, f"pca_pc{i+1}_time_course.png")
        plt.savefig(time_course_path)
        plt.close()
        save_thumbnail(time_course_path)
        print(f"PC {i+1} time course plot saved to {time_course_path}")

        # Spatial map plot
        plt.figure(figsize=(8, 5))
        component_weights = np.abs(components[i])
        component_map = np.zeros(roi_masks.shape[:2])
        for cell_idx in range(roi_masks.shape[2]):
            if cell_idx < len(component_weights):
                weight = component_weights[cell_idx]
                roi_mask = roi_masks[..., cell_idx]
                component_map[roi_mask > 0] = weight
        
        plt.imshow(component_map, cmap="viridis")
        plt.colorbar(label="Component Weight")
        plt.title(f"PC {i+1} Spatial Map")
        plt.tight_layout()
        spatial_map_path = os.path.join(output_dir, f"pca_pc{i+1}_spatial_map.png")
        plt.savefig(spatial_map_path)
        plt.close()
        save_thumbnail(spatial_map_path)
        print(f"PC {i+1} spatial map plot saved to {spatial_map_path}")

# Updated K-means visualization function
def generate_kmeans_visualization(labels, corr_matrix, fluorescence, roi_masks, output_dir):
    """
    Generate standalone K-means visualizations
    """
    # Reorder correlation matrix based on clusters
    sort_idx = np.argsort(labels)
    sorted_corr_matrix = corr_matrix[sort_idx][:, sort_idx]
    
    # Calculate cluster information
    unique_clusters = np.unique(labels)
    n_clusters = len(unique_clusters)
    colors = plt.cm.jet(np.linspace(0, 1, n_clusters)) 
    custom_cmap = ListedColormap(colors)
    
    # Calculate mean time course for each cluster
    cluster_averages = []
    for cluster in unique_clusters:
        cluster_mask = labels == cluster
        cluster_avg = np.mean(fluorescence[cluster_mask], axis=0)
        cluster_averages.append(cluster_avg)
    
    # Generate cluster map
    cluster_map = np.zeros(roi_masks.shape[:2])
    for i, label in enumerate(labels):
        if i < roi_masks.shape[2]:
            roi_mask = roi_masks[..., i]
            cluster_map[roi_mask > 0] = label + 1
    
    # Plot correlation matrix
    plt.figure(figsize=(8, 5))
    plt.imshow(sorted_corr_matrix, cmap="jet")
    plt.colorbar(label="Correlation")
    plt.title(f"K-means Clustering (k={n_clusters})")
    plt.xlabel("Cells")
    plt.ylabel("Cells")
    plt.tight_layout()
    corr_matrix_path = os.path.join(output_dir, "kmeans_correlation_matrix.png")
    plt.savefig(corr_matrix_path)
    plt.close()
    save_thumbnail(corr_matrix_path)
    print(f"Correlation matrix plot saved to {corr_matrix_path}")

    # Plot average time courses
    plt.figure(figsize=(8, 5))
    for i, avg in enumerate(cluster_averages):
        plt.plot(avg, color=colors[i], linewidth=2, label=f"Cluster {i+1}")
    plt.title("Mean Time Course by Cluster")
    plt.xlabel("Time")
    plt.ylabel("Fluorescence")
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    time_course_path = os.path.join(output_dir, "kmeans_time_courses.png")
    plt.savefig(time_course_path)
    plt.close()
    save_thumbnail(time_course_path)
    print(f"Time course plot saved to {time_course_path}")

    # Plot cluster map
    plt.figure(figsize=(8, 5))
    plt.imshow(cluster_map, cmap=custom_cmap)
    plt.colorbar(label="Cluster")
    plt.title("Cluster Assignments")
    plt.tight_layout()
    cluster_map_path = os.path.join(output_dir, "kmeans_cluster_map.png")
    plt.savefig(cluster_map_path)
    plt.close()
    save_thumbnail(cluster_map_path)
    print(f"Cluster map plot saved to {cluster_map_path}")

# Ensure output directory exists
output_dir = "./test_plots"
os.makedirs(output_dir, exist_ok=True)

# Test PCA visualization
pca_scores = score  # Replace with actual data
pca_explained_variance = explained  # Replace with actual data
pca_components = coeff.T  # Replace with actual data
roi_masks = np.zeros((labelimage.shape[0], labelimage.shape[1], int(np.max(labelimage))))
for cell_idx in range(1, int(np.max(labelimage)) + 1):
    roi_masks[:, :, cell_idx-1] = (labelimage == cell_idx)

generate_pca_visualization(pca_scores, pca_explained_variance, pca_components, roi_masks, output_dir)

# Test K-means visualization
kmeans_labels = idx_kmeans  # Replace with actual data
kmeans_corr_matrix = corr_matrix  # Replace with actual data
kmeans_fluorescence = timecourse  # Replace with actual data

generate_kmeans_visualization(kmeans_labels, kmeans_corr_matrix, kmeans_fluorescence, roi_masks, output_dir)